In [1]:
import torch
import torch.optim as optim
import numpy as np
from transformer import Transformer
from metrics import Evaluator
from datasets import load_dataset
from utils import *
from training import *
from tokenizer import *

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device}")

/Users/tonimo/Documents/Developing/Repos/Deep-Learning/deep-learning-venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using cpu


In [2]:
traindict = load_dataset("opus100", "de-en", split="train")
testdict = load_dataset("opus100", "de-en", split="test")

Found cached dataset opus100 (/Users/tonimo/.cache/huggingface/datasets/opus100/de-en/0.0.0/256f3196b69901fb0c79810ef468e2c4ed84fbd563719920b1ff1fdc750f7704)
Found cached dataset opus100 (/Users/tonimo/.cache/huggingface/datasets/opus100/de-en/0.0.0/256f3196b69901fb0c79810ef468e2c4ed84fbd563719920b1ff1fdc750f7704)


In [3]:
train_inputs, train_labels = get_split(traindict, "en", "de", size=100)
test_inputs, test_labels = get_split(testdict, "en", "de", size=10)
trainset = Dataset(train_inputs, train_labels)
testset = Dataset(test_inputs, test_labels)

In [4]:
trainframe = trainset.dataframe()
trainframe.head()

,inputs,labels
0,It's greed that it's gonna be the death of you...,Deine Habgier wird noch dein Tod sein.
1,Vega.,- Vega.
2,Just say when.,Sagen Sie einfach stopp.
3,- Wait.,- Warte.
4,I don't wanna be here.,Ich will nicht hier sein.


In [5]:
testframe = testset.dataframe()
testframe.head()

,inputs,labels
0,"By clicking on 'Save profile', you the user ag...",Die Nutzungsbedingungen werden durch das Klick...
1,I wanted to show you something first.,Ich wollte dir erst noch etwas zeigen.
2,You have suffered because of Shinkichi.,Du musstest wegen Shinkichi leiden.
3,"moodle:bg-bab: Calendar: Day view: Friday, 25 ...",moodle:bg-bab: Kalender: Tagesansicht: Freitag...
4,"I mean, most people, they see another person w...","Ich meine, die meisten Leuten sehen eine ander..."


In [6]:
print(trainframe.isnull().values.any())
trainframe.describe()

False


,inputs,labels
count,100,100
unique,100,100
top,It's greed that it's gonna be the death of you...,Deine Habgier wird noch dein Tod sein.
freq,1,1


In [7]:
print(testframe.isnull().values.any())
testframe.describe()

False


,inputs,labels
count,10,10
unique,10,10
top,"By clicking on 'Save profile', you the user ag...",Die Nutzungsbedingungen werden durch das Klick...
freq,1,1


In [8]:
trainset.sample()

[("It's greed that it's gonna be the death of you, 'cause you...",
  'Deine Habgier wird noch dein Tod sein.')]

In [9]:
testset.sample()

[('I wanted to show you something first.',
  'Ich wollte dir erst noch etwas zeigen.')]

In [10]:
corpus_en = trainset.corpus(split=0) + testset.corpus(split=0)
corpus_de = trainset.corpus(split=1) + testset.corpus(split=1)
tokenizer_en = Nerdimizer()
tokenizer_de = Nerdimizer()
tokenizer_en.train(corpus_en, size=50000)
tokenizer_de.train(corpus_de, size=50000)
translator = Translator(tokenizer_en, tokenizer_de)

In [11]:
en_vocab, de_vocab = translator.vocab_size()
maxlen_train = trainset.maxlen(translator)
maxlen_test = testset.maxlen(translator)
maxlen = min(maxlen_train, maxlen_test, 200)
start, end, pad = tokenizer_en["[S]"], tokenizer_en["[E]"], tokenizer_en["[P]"]
print(f"Number of input tokens: {len(tokenizer_en)}\nNumber of output tokens: {len(tokenizer_de)}")

Number of input tokens: 861
Number of output tokens: 1064


In [12]:
translator.padon(maxlen, end=True, pad_id=pad)
translator.truncon(maxlen, end=True)
print(f"Maxlen: {maxlen}")

Maxlen: 46


In [13]:
tokenized_trainset = trainset.tokenized(translator, model=True)
dataloader = tokenized_trainset.dataloader(batch_size=32, drop_last=False)

In [14]:
model = Transformer(en_vocab, de_vocab, maxlen, pad_id=pad, dm=256, nhead=8, layers=3, dff=1024)
optimizer = optim.Adam(model.parameters(), lr=1e-5, betas=(0.9, 0.98), eps=10e-9)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.9, patience=10)
evaluator = Evaluator(testset, translator, "[S]", "[E]", maxlen, sample=10, ngrams=4, threshold=20, 
                    mode="geometric", device=device)
clock = Clock()
checkpoint = Checkpoint(model, optimizer, scheduler, evaluator, clock, epochs=3, 
                    path="checkpoints/english-german", overwrite=False)
model.to(device);

In [15]:
train(dataloader, model, optimizer, scheduler, evaluator, checkpoint, clock,
    epochs=5, warmups=100, verbose=True, device=device)

Training Started
Counter({'ol': 36, '[S]anciancianci': 1, 'olgegegegege': 1})

Counter({'der': 3, 'die': 2, '[S]': 1, 'verfahren': 1, 'auf': 1, 'grundlage': 1, 'des': 1, 'rhe': 1, 'modells': 1, 'erlauben': 1, 'messung': 1, 'ereignisse': 1, 'zu': 1, 'beginn': 1, 'kaskade': 1, '.': 1, '[E]': 1})
Counter({'ol ol': 34, '[S]anciancianci ol': 1, 'ol olgegegegege': 1, 'olgegegegege ol': 1})

Counter({'[S] die': 1, 'die verfahren': 1, 'verfahren auf': 1, 'auf der': 1, 'der grundlage': 1, 'grundlage des': 1, 'des rhe': 1, 'rhe modells': 1, 'modells erlauben': 1, 'erlauben die': 1, 'die messung': 1, 'messung der': 1, 'der ereignisse': 1, 'ereignisse zu': 1, 'zu beginn': 1, 'beginn der': 1, 'der kaskade': 1, 'kaskade .': 1, '. [E]': 1})
Counter({'ol ol ol': 32, '[S]anciancianci ol ol': 1, 'ol ol olgegegegege': 1, 'ol olgegegegege ol': 1, 'olgegegegege ol ol': 1})

Counter({'[S] die verfahren': 1, 'die verfahren auf': 1, 'verfahren auf der': 1, 'auf der grundlage': 1, 'der grundlage des': 1, 'grun

6.7997398853302

In [16]:
checkpoint = Checkpoint(model, optimizer, scheduler)
checkpoint.load_checkpoint(path="checkpoints/english-german-3")

Checkpoint Loaded


In [17]:
retrain(dataloader, checkpoint, 11)

Training Resumed
Counter({'[S]': 1, '[E]': 1})

Counter({'[S]': 1, 'ich': 1, 'wollte': 1, 'dir': 1, 'erst': 1, 'noch': 1, 'etwas': 1, 'zeigen': 1, '.': 1, '[E]': 1})
Counter({'[S] [E]': 1})

Counter({'[S] ich': 1, 'ich wollte': 1, 'wollte dir': 1, 'dir erst': 1, 'erst noch': 1, 'noch etwas': 1, 'etwas zeigen': 1, 'zeigen .': 1, '. [E]': 1})
Counter()

Counter({'[S] ich wollte': 1, 'ich wollte dir': 1, 'wollte dir erst': 1, 'dir erst noch': 1, 'erst noch etwas': 1, 'noch etwas zeigen': 1, 'etwas zeigen .': 1, 'zeigen . [E]': 1})
Counter()

Counter({'[S] ich wollte dir': 1, 'ich wollte dir erst': 1, 'wollte dir erst noch': 1, 'dir erst noch etwas': 1, 'erst noch etwas zeigen': 1, 'noch etwas zeigen .': 1, 'etwas zeigen . [E]': 1})
{'bleu': 0.0, 'percisions': [100.0, 0.0, 0, 0], 'prediction length': 2, 'reference length': 10}
Counter({'[S]': 1, '[E]': 1})

Counter({'[S]': 1, 'du': 1, 'musstest': 1, 'wegen': 1, 'shinkichi': 1, 'leiden': 1, '.': 1, '[E]': 1})
Counter({'[S] [E]': 1})

Counte

5.17160701751709